In [127]:
import requests
from bs4 import BeautifulSoup
import re
import csv
from csv import writer
import random
import time
import pandas as pd
import csv
import numpy as np
import math
import itertools
import concurrent.futures
import datetime
import datetime
from datetime import date

In [128]:
df = pd.read_csv('mockPreds.csv')
df.head()

,Fight,Prediction,Predicted,Winner,Result
0,Edson Barboza vs Lerone Murphy,0.585,Lerone Murphy,NaN,NaN
1,Khaos Williams vs Carlston Harris,0.400,Khaos Willaims,NaN,NaN
2,Themba Gorimbo vs Ramiz Brahimaj,0.550,Ramiz Brahimaj,NaN,NaN
3,Adrian Yanez vs Vinicius Salvador,0.720,Vinicius Salvador,NaN,NaN


In [129]:
dfCheck = pd.read_csv('databaseUpdated2024-05-22.csv')
dfCheck.head()

,fight,redCorner,blueCorner,winner,event,referee,method_of_victory,date,venue,title_fight,...,redCorner_subs_attempted,blueCorner_subs_attempted,round,time,redCorner_height,blueCorner_height,redCorner_reach,blueCorner_reach,redCorner_stance,blueCorner_stance
0,Edson Barboza vs Lerone Murphy,Edson Barboza,Lerone Murphy,Lerone Murphy,UFC Fight Night: Barboza vs. Murphy,Herb Dean,Decision - Unanimous,05.18.2024,UFC Apex,no,...,0,0,5,5:00,"5' 11""","5' 9""","75""","73""",Orthodox,Orthodox
1,Khaos Williams vs Carlston Harris,Khaos Williams,Carlston Harris,Khaos Williams,UFC Fight Night: Barboza vs. Murphy,Dan Miragliotta,KO/TKO,05.18.2024,UFC Apex,no,...,0,0,1,1:30,"6' 0""","6' 0""","77""","76""",Orthodox,Orthodox
2,Ramiz Brahimaj vs Themba Gorimbo,Themba Gorimbo,Ramiz Brahimaj,Themba Gorimbo,UFC Fight Night: Barboza vs. Murphy,Mark Smith,Decision - Unanimous,05.18.2024,UFC Apex,no,...,0,0,3,5:00,"6' 1""","5' 10""","77""","72""",Orthodox,Orthodox
3,Adrian Yanez vs Vinicius Salvador,Adrian Yanez,Vinicius Salvador,Adrian Yanez,UFC Fight Night: Barboza vs. Murphy,Chris Tognoni,KO/TKO,05.18.2024,UFC Apex,no,...,0,0,1,2:47,"5' 7""","5' 7""","70""","70""",Orthodox,Southpaw
4,Angela Hill vs Luana Pinheiro,Luana Pinheiro,Angela Hill,Angela Hill,UFC Fight Night: Barboza vs. Murphy,Herb Dean,Submission,05.18.2024,UFC Apex,no,...,0,1,2,4:12,"5' 2""","5' 3""","62""","64""",Orthodox,Orthodox


In [130]:
#find index of the last fight of the previous event
change = dfCheck['event'] != dfCheck['event'].shift(1)

# Find the first index where the 'event' is different
indexDifferent = dfCheck[change].index[1] 

limitOfCheck = indexDifferent-1

print(limitOfCheck)



11


In [131]:
for index, row in df.iterrows():
    winner = None
    if(type(row['Result']) != str):
        fight = row['Fight']
        redAndBlue = fight.split(' vs ')
        redCorner = redAndBlue[0]
        blueCorner = redAndBlue[1]
        prediction = row['Prediction']
        #find indexes of fight match
        try:
            redCornerCheck = dfCheck[:limitOfCheck][dfCheck['redCorner'][:limitOfCheck] == redCorner].index[0]
            blueCornerCheck = dfCheck[:limitOfCheck][dfCheck['blueCorner'][:limitOfCheck] == blueCorner].index[0]
            if(int(redCornerCheck) == int(blueCornerCheck)):
                indexCheck = int(redCornerCheck)
                winner = dfCheck.loc[indexCheck, 'winner']
                if(winner == redCorner and float(prediction) < 0.5):
                    df.loc[index, 'Result'] = 'Correct'
                    df.loc[index, 'Winner'] = winner
                elif(winner == blueCorner and float(prediction) > 0.5):
                    df.loc[index, 'Result'] = 'Correct'
                    df.loc[index, 'Winner'] = winner
                else:
                    df.loc[index, 'Result'] = 'Incorrect'
                    df.loc[index, 'Winner'] = winner
        except:
            #upon exception, results get scraped as names will match here
            url = 'https://www.espn.com/mma/schedule/_/league/ufc'
            headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
    
            #site request
            site = requests.get(url, headers=headers)
            soup = BeautifulSoup(site.content, 'html.parser')  

            #get table
            tbodys = soup.find_all('tbody', class_=re.compile('Table__TBODY')) 

            #get part
            table = tbodys[len(tbodys)-1]
            links = table.find_all('a', class_=re.compile('AnchorLink'))
            lastEvent = links[0]
            part = lastEvent['href']

            url = f'https://www.espn.com{part}'
            headers = {'User-Agent': "insomnia/9.1.1"}
    
            #site request
            site = requests.get(url, headers=headers)
            soup = BeautifulSoup(site.content, 'html.parser')

            right = soup.find_all('div', class_=re.compile('MMACompetitor relative flex flex-uniform pl6 MMACompetitor--desktop'))
            left = soup.find_all('div', class_=re.compile('MMACompetitor relative flex flex-uniform pr6 flex-row-reverse MMACompetitor--desktop'))

            #find match
            for item1, item2 in zip(left, right):
                if redCorner in item1.text.strip() and blueCorner in item2.text.strip():
                    if(item1.find('svg') is not None):
                        winner = redCorner
                    if(item2.find('svg') is not None):
                        winner = blueCorner
            if(winner == redCorner and float(prediction) < 0.5):
                df.loc[index, 'Result'] = 'Correct'
                df.loc[index, 'Winner'] = winner
            elif(winner == blueCorner and float(prediction) > 0.5):
                df.loc[index, 'Result'] = 'Correct'
                df.loc[index, 'Winner'] = winner
            else:
                df.loc[index, 'Result'] = 'Incorrect'
                df.loc[index, 'Winner'] = winner

df.head()
            
            





/var/folders/pw/f4_6q0l977bcqprlylly2d1w0000gq/T/ipykernel_56563/975752863.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Correct' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'Result'] = 'Correct'
/var/folders/pw/f4_6q0l977bcqprlylly2d1w0000gq/T/ipykernel_56563/975752863.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Lerone Murphy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'Winner'] = winner


,Fight,Prediction,Predicted,Winner,Result
0,Edson Barboza vs Lerone Murphy,0.585,Lerone Murphy,Lerone Murphy,Correct
1,Khaos Williams vs Carlston Harris,0.400,Khaos Willaims,Khaos Williams,Correct
2,Themba Gorimbo vs Ramiz Brahimaj,0.550,Ramiz Brahimaj,Themba Gorimbo,Incorrect
3,Adrian Yanez vs Vinicius Salvador,0.720,Vinicius Salvador,Adrian Yanez,Incorrect
